In [1]:
"""
generate random fits files satisfying certain criteria and
upload them to an s3 bucket
"""
from multiprocessing import Pool

from killscreen.aws.s3 import Bucket
from killscreen.aws.utilities import make_boto_session

from s3_fuse.random_generators import fits_file
from s3_fuse.bench_config import CASES, SHAPES, GENERATORS
from s3_fuse.transfer_config import (
    TRANSFERCONFIG, AWS_IAM_SECRETS_FILE, AWS_REGION
)

BUCKET_NAME = "bucket_sadly_bare_of_fits_files"

In [ ]:
# mock data generating & uploading function
def generate_and_upload(case, prefix, quantity):
    shape, generator, dtype, hdu_count, compression_type = case
    bucket = Bucket(
        BUCKET_NAME,
        session=make_boto_session(credential_file=AWS_IAM_SECRETS_FILE, region=AWS_REGION),
        config=TRANSFERCONFIG
    )
    for ix in range(quantity):
        print(case, ix)
        fits_blob = fits_file(
            size=SHAPES[shape],
            dtype=dtype,
            element_generator=GENERATORS[generator],
            base_hdu_count=hdu_count,
            compression_type=compression_type,
            return_bytes=True
        )
        bucket.put(fits_blob, f"{prefix}/{prefix}_{ix}.fits")

In [ ]:
# TODO: add ls-type functionality to check existing file counts
quantity = 10
pool = Pool(5)
results = {}

# asynchronously map our gen-and-upload function across cases defined in s3_fuse.bench_config
for prefix, case in CASES.items():
    results[prefix] = pool.apply_async(
        generate_and_upload, (case, prefix, quantity)
    )
pool.close()
pool.join()